<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/yvp_vegetation_cover_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*R Notebook*

# README

* [Readme fixed grid plot vegetation data](https://docs.google.com/document/d/16-Aq8u9Rudd78fSzfjvpCXyQgE-BstC-d2PjYfmLtcw/edit?usp=sharing)

# Load Tools

In [ ]:
# Package and library installation
packages_needed = c("tidyverse", "gsheet", "lubridate") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

In [ ]:
# Package and library installation
packages_needed = c("bigrquery") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

# Source

## YVP vegetation cover

In [ ]:
# 2020-04-28_yvp_vegetation_cover
src = 'https://drive.google.com/uc?id=1pemnlKIlfAQw2JSMN7yDlYMG5QhUW-NP'

In [ ]:
df <- read.csv(file = src)

In [ ]:
head(df, n=2)

,plot_code,date,subplot,species_code,cover_pct
,<chr>,<chr>,<int>,<chr>,<int>
1,YVP 10,2017-06-09,1,BOESPP,1
2,YVP 10,2017-06-09,1,CREINT,1


## Plant species metadata

In [ ]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")
billing <- bq_test_project()

In [ ]:
spe_sq <- "
            SELECT *
            FROM `mpg-data-warehouse.vegetation_species_metadata.vegetation_species_metadata`
            "
spe_bq <- bq_project_query(billing, spe_sq)
spe_tb <- bq_table_download(spe_bq)
spe_df <- as.data.frame(spe_tb) %>% glimpse()

Rows: 754
Columns: 9
$ key_plant_species   <int> 360, 13, 26, 53, 738, 75, 76, 746, 83, 88, 86, 87…
$ key_plant_code      <chr> "NV", "AGRSCA", "ANDGER", "ARIPUR", "BOUCUR", "BO…
$ plant_name_sci      <chr> "no vegetation", "Agrostis scabra", "Andropogon g…
$ plant_name_syn      <chr> NA, NA, NA, "Aristida longiseta", NA, NA, NA, NA,…
$ plant_name_common   <chr> "no vegetation", "rough bentgrass", "big bluestem…
$ plant_name_family   <chr> "None", "Poaceae", "Poaceae", "Poaceae", "Poaceae…
$ plant_native_status <chr> "none", "native", "native", "native", "native", "…
$ plant_life_cycle    <chr> "unknown", "perennial", "perennial", "perennial",…
$ plant_life_form     <chr> "none", "graminoid", "graminoid", "graminoid", "g…


# Wrangle

## Structure columns

## Plot Code Transformation
The plot code used in the source data is a complex string. It is needed to provide a unique key to each survey location, but because it is a string it is difficult to sort or filter plots. Further, the plot codes used here will be difficult to associate with the extensive grid point metadata stored elsewhere in the MPG Data Warehouse. 

Solution: paste the separate identifers from the plot code into separate fields, but retain the original character string for internal use.

#### plot_code

In [ ]:
# convert to string
df$plot_code <- as.character(df$plot_code)

#### plot_ loc

In [ ]:
# detect "N" in 'plot_code' and write to new column 'plot_loc'
df <- df %>%
  mutate(plot_loc = ifelse(str_detect(plot_code, "N"), "N", NA))

In [ ]:
# reorder columns
df <- df[,c(1,6,2,3,4,5)]

#### plot_rep

In [ ]:
# detect "A", "B", "C" characters in plot_code and if present write to 'plot_rep'
df <- df %>%
  mutate(plot_rep = case_when(str_detect(plot_code, "A")~"A",
                              str_detect(plot_code, "B")~"B",
                              str_detect(plot_code, "C")~"C"))

In [ ]:
# reorder columns
df <- df[,c(1,2,7,3,4,5,6)]

#### plot_num

In [ ]:
# use digital values from 'plot_code' and to populate 'plot_num'
df <- df %>%
  mutate(plot_num = str_extract(plot_code, "[:digit:].*"),
         plot_num = as.integer(plot_num))

In [ ]:
# reorder columns
df <- df[,c(1,2,3,8,4,5,6,7)]

### date
* Convert to date
* Add a year varaible to assist with wrangling later

In [ ]:
# convert to date
df$date <- as.Date(df$date)
df$year <- year(df$date)

In [ ]:
df %>% glimpse()

Rows: 21,728
Columns: 9
$ plot_code    <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 1…
$ plot_loc     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ plot_rep     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ plot_num     <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, …
$ date         <date> 2017-06-09, 2017-06-09, 2017-06-09, 2017-06-09, 2017-06…
$ subplot      <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ species_code <chr> "BOESPP", "CREINT", "EUPESU", "FESCAM", "FESIDA", "GEUTR…
$ cover_pct    <int> 1, 1, 5, 25, 25, 10, 1, 1, 5, 1, 30, 2, 2, 1, 3, 0, 1, 0…
$ year         <dbl> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 20…


### subplot

In [ ]:
# convert to integer
df$subplot <- as.integer(df$subplot)

### species_key

This will be imported from the plant species metadata table, and we can use it to join and correct species codes in the future. But because joining the key to the species codes will require that the codes be corrected first, we will skip this step for now.

Elsewhere in the warehouse, this variable is named `key_plant_species`, so we will maintain this convention here as well. 


### species_code

Elsewhere in the warehouse, species_code is `key_plant_code`, wo we will maintain that convention here too. 

In [ ]:
# convert to string
df$species_code <- as.character(df$species_code)

In [ ]:
df <-
df %>% 
mutate(key_plant_code = species_code) %>% 
select(-species_code)

## Identify Double Counted Species
In a few instances, a plant species is counted twice in the same survey subplot. This could inflate the cover reported for that species. In these cases, the desired end product is to have just one row for each. Because there is no way to know which value of the two is correct, we will average the two values, treating them empirically as independent, legitimate cover estimates. 

In [ ]:
df %>% summary()

  plot_code           plot_loc           plot_rep            plot_num    
 Length:21728       Length:21728       Length:21728       Min.   :  7.0  
 Class :character   Class :character   Class :character   1st Qu.: 62.0  
 Mode  :character   Mode  :character   Mode  :character   Median :209.0  
                                                          Mean   :244.8  
                                                          3rd Qu.:386.0  
                                                          Max.   :571.0  
      date               subplot         cover_pct            year     
 Min.   :2017-05-08   Min.   : 1.000   Min.   :  0.000   Min.   :2017  
 1st Qu.:2017-06-09   1st Qu.: 3.000   1st Qu.:  1.000   1st Qu.:2017  
 Median :2018-07-02   Median : 5.000   Median :  2.000   Median :2018  
 Mean   :2018-07-22   Mean   : 5.497   Mean   :  6.691   Mean   :2018  
 3rd Qu.:2019-05-28   3rd Qu.: 8.000   3rd Qu.:  5.000   3rd Qu.:2019  
 Max.   :2019-07-16   Max.   :10.000   Max.   :100

No NA values in the quantitative data. Data contains 21728 rows. 

In [ ]:
df %>%
group_by(year = as.integer(year(df$date)), plot_code, subplot, key_plant_code) %>%
summarize(counted = n()) %>% 
ungroup() %>%
arrange(year, plot_code, subplot, desc(counted)) %>%
filter(counted > 1) %>%
print(n=Inf)

`summarise()` regrouping output by 'year', 'plot_code', 'subplot' (override with `.groups` argument)



# A tibble: 46 x 5
    year plot_code subplot key_plant_code counted
   <int> <chr>       <int> <chr>            <int>
 1  2017 YVP 144         2 VERVER               2
 2  2017 YVP 180         7 FRIPUD               2
 3  2017 YVP 203         4 COLLIN               2
 4  2017 YVP 355        10 PSESPI               2
 5  2017 YVP 44          9 ORTTEN               2
 6  2017 YVP N111        2 DRAVER               2
 7  2017 YVP NB294       8 MICGRA               2
 8  2018 YVP 112         9 ALYALY               2
 9  2018 YVP 12          4 HOLUMB               2
10  2018 YVP 144        10 ACHMIL               2
11  2018 YVP 184         4 HOLUMB               2
12  2018 YVP 185         1 DRAVER               2
13  2018 YVP 185         1 FRIPUD               2
14  2018 YVP 185         1 MYOMIC               2
15  2018 YVP 185         3 HOLUMB               2
16  2018 YVP 185         4 ARANUT               2
17  2018 YVP 185         4 EPIBRA               2
18  2018 YVP 20          2 VERV

In [ ]:
df <-
  df %>%
  group_by(year, plot_code, plot_loc, plot_rep, plot_num, date, subplot, key_plant_code) %>%
  summarize(cover_pct = mean(cover_pct)) %>% 
  ungroup() %>% glimpse()

`summarise()` regrouping output by 'year', 'plot_code', 'plot_loc', 'plot_rep', 'plot_num', 'date', 'subplot' (override with `.groups` argument)



Rows: 21,682
Columns: 9
$ year           <dbl> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, …
$ plot_code      <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP…
$ plot_loc       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ plot_rep       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ plot_num       <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10…
$ date           <date> 2017-06-09, 2017-06-09, 2017-06-09, 2017-06-09, 2017-…
$ subplot        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, …
$ key_plant_code <chr> "BOESPP", "CREINT", "EUPESU", "FESCAM", "FESIDA", "GEU…
$ cover_pct      <dbl> 1, 1, 5, 25, 25, 10, 1, 1, 5, 1, 30, 2, 2, 1, 4, 5, 40…


After averaging the doubles, the data frame contains 21682 rows, and there were 46 double counted species so that looks correct. Histograms and other diagnostics were produced in a scratch cell, and the process seems legitimate. 

## Correct errors in species codes
The species codes used in the source data contain numerous errors, and they also in some cases represent old taxonomy where species names have been revised. This can cause all sorts of problems, like artificially creating new species or making it impossible to join with available species metadata. Several steps must be accomplished here:

1. Trim leading or trailing spaces from the code (this was done in excel before source CSV files were created)
2. Read in master list of species metadata and query YVP species codes to identify which ones don't align
3. Align the species codes, identify the ones that are wrong and correct them
4. Import the numeric key from the species metadata so that future aligments are easier and errors are less common

### Align species codes and identify mistakes


In [ ]:
# Align the species codes 
# Produce df of codes that don't match the master list
collisions_species_codes <- 
  df %>% 
  anti_join(spe_df, by = c("key_plant_code" = "key_plant_code")) %>% 
  group_by(key_plant_code) %>% 
  distinct(key_plant_code) %>% 
  arrange(key_plant_code) %>% 
  print(n = Inf)

# A tibble: 60 x 1
# Groups:   key_plant_code [60]
   key_plant_code     
   <chr>              
 1 AGOS SP            
 2 AGROSP             
 3 ALOP SP            
 4 ANDOCCUAL          
 5 ANTE SP            
 6 ANTSPP             
 7 ARNCOR?            
 8 ARTE SP            
 9 ARTSPP             
10 BOEC SP            
11 BOESPP             
12 CARE SP            
13 CARE SP2           
14 CARE SP4           
15 CARSPP             
16 CARSPP 1           
17 CARSPP 2           
18 CARSPP 4           
19 CARSPP2            
20 CASSPP             
21 CAST SP            
22 CERA SP            
23 CERINT             
24 CHEN SP            
25 CREIINT            
26 CREP SP            
27 CRYP SP            
28 DESC SP            
29 DODE SP            
30 DODOUL             
31 DODSPP             
32 DODSSP             
33 DRASPP             
34 ELYM SP            
35 ELYSPP             
36 EPISPP             
37 EQUI SP            
38 EQUSPP             
39 ERIG SP            
40 ERI

### Create file that associates errors with corrections

In [ ]:
# Produce file `collisions_species_codes` for work in spreadsheet outside of this environment
# The file will save to the `content` folder in the drive tree
# BL downloaded the file to his desktop to produce a new naming key file
filename = "collisions_species_codes.csv"
if (filename %in% list.files(getwd())) {
  cat("file already exists in working directory: ", filename, "\n", "working directory: ", getwd(), "\n")
} else {
  write.csv(collisions_species_codes, filename)
  cat(filename, " written to working directory \n", "working directory: ", getwd(), "\n")
}


collisions_species_codes.csv  written to working directory 
 working directory:  /content 


In [ ]:
# Import csv file with the updated codes 
# This file was produced by visually aligning the codes with a file that Rebecca Durham provided
code_corrections <- read.csv(file = "https://drive.google.com/uc?id=1D0j3U4Or2PviFS02F3rxTRXr1SpGOB0a",
  colClasses = c("character", "character")) %>% 
glimpse()

Rows: 60
Columns: 2
$ plantcode_incorrect <chr> "AGOS SP", "ALOP SP", "ANDOCCUAL", "ARNCOR?", "AR…
$ plantcode_corrected <chr> "AGOS_SP", "ALOP_SP", "ANDOCC", "ARNCOR", "ARTDRA…


### Cascade changes through dataset


In [ ]:
# Create new df to hold corrected information
# Change species_code to character variable to avoid problems with levels later
yvp_veg_cover_correct <- 
  df %>% 
  mutate(key_plant_code = as.character(key_plant_code)) %>% 
  select(-year) %>% 
  glimpse()

Rows: 21,682
Columns: 8
$ plot_code      <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP…
$ plot_loc       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ plot_rep       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…
$ plot_num       <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10…
$ date           <date> 2017-06-09, 2017-06-09, 2017-06-09, 2017-06-09, 2017-…
$ subplot        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, …
$ key_plant_code <chr> "BOESPP", "CREINT", "EUPESU", "FESCAM", "FESIDA", "GEU…
$ cover_pct      <dbl> 1, 1, 5, 25, 25, 10, 1, 1, 5, 1, 30, 2, 2, 1, 4, 5, 40…


In [ ]:
# Loop operation used to update each instance of an incorrect code
# Embed logic control to prevent errors if this loop is run on a df with corrected codes
# Variable to track loop cycles
cycles = 0

for (i in 1:length(code_corrections[, 1])) {
  index = which(yvp_veg_cover_correct$key_plant_code == code_corrections$plantcode_incorrect[i])

  if (length(index != 0)) {
    cat("number of incorrect code entries: ", length(index), "\n")
    cat("incorrect code: ", code_corrections$plantcode_incorrect[i], "\n")
    yvp_veg_cover_correct[index, ]$key_plant_code = code_corrections$plantcode_corrected[i]
    print(yvp_veg_cover_correct[index, c(1,5,6,7,8)])
    cycles = cycles + length(index)
    cat("\n")
  } else {
    cat("no incorrect code entries were found \n")
  }

  cat("number of corrections made (cumulative): ", cycles, "\n\n\n")

}

number of incorrect code entries:  1 
incorrect code:  AGOS SP 
# A tibble: 1 x 5
  plot_code date       subplot key_plant_code cover_pct
  <chr>     <date>       <int> <chr>              <dbl>
1 YVP 205   2019-06-26       4 AGOS_SP                1

number of corrections made (cumulative):  1 


number of incorrect code entries:  9 
incorrect code:  ALOP SP 
# A tibble: 9 x 5
  plot_code date       subplot key_plant_code cover_pct
  <chr>     <date>       <int> <chr>              <dbl>
1 YVP N348  2019-07-05       2 ALOP_SP               40
2 YVP N348  2019-07-05       3 ALOP_SP               10
3 YVP N348  2019-07-05       4 ALOP_SP               50
4 YVP N348  2019-07-05       5 ALOP_SP               80
5 YVP N348  2019-07-05       6 ALOP_SP               10
6 YVP N348  2019-07-05       7 ALOP_SP               65
7 YVP N348  2019-07-05       8 ALOP_SP               25
8 YVP N348  2019-07-05       9 ALOP_SP               60
9 YVP N348  2019-07-05      10 ALOP_SP                2

num

436 cumulative corrections as of 2020-11-06

In [ ]:
# Rescan for incorrect species codes
yvp_veg_cover_correct %>% 
anti_join(spe_df, by = c("key_plant_code" = "key_plant_code")) %>% 
group_by(key_plant_code) %>% distinct(key_plant_code) %>% arrange(key_plant_code)

key_plant_code
<chr>


## Incorporate serial key for species codes

In [ ]:
yvp_vegetation_cover_FINAL <- 
  yvp_veg_cover_correct %>% 
  left_join(spe_df %>% select(key_plant_species, key_plant_code), by = c("key_plant_code" = "key_plant_code")) %>% 
  select(c(1,2,3,4,5,6,9,7,8)) %>% 
  rename(grid_point = plot_num) %>% 
  glimpse()

Rows: 21,682
Columns: 9
$ plot_code         <chr> "YVP 10", "YVP 10", "YVP 10", "YVP 10", "YVP 10", "…
$ plot_loc          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ plot_rep          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ grid_point        <int> 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,…
$ date              <date> 2017-06-09, 2017-06-09, 2017-06-09, 2017-06-09, 20…
$ subplot           <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, …
$ key_plant_species <int> 780, 163, 230, 232, 233, 250, 84, 316, 320, 343, 48…
$ key_plant_code    <chr> "BOEC_SP", "CREINT", "EUPESU", "FESCAM", "FESIDA", …
$ cover_pct         <dbl> 1, 1, 5, 25, 25, 10, 1, 1, 5, 1, 30, 2, 2, 1, 4, 5,…


In [ ]:
summary(yvp_vegetation_cover_FINAL)

  plot_code           plot_loc           plot_rep           grid_point   
 Length:21682       Length:21682       Length:21682       Min.   :  7.0  
 Class :character   Class :character   Class :character   1st Qu.: 62.0  
 Mode  :character   Mode  :character   Mode  :character   Median :209.0  
                                                          Mean   :244.9  
                                                          3rd Qu.:386.0  
                                                          Max.   :571.0  
      date               subplot       key_plant_species key_plant_code    
 Min.   :2017-05-08   Min.   : 1.000   Min.   :  3.0     Length:21682      
 1st Qu.:2017-06-09   1st Qu.: 3.000   1st Qu.:153.0     Class :character  
 Median :2018-07-02   Median : 5.000   Median :274.0     Mode  :character  
 Mean   :2018-07-22   Mean   : 5.499   Mean   :280.1                       
 3rd Qu.:2019-05-28   3rd Qu.: 8.000   3rd Qu.:411.0                       
 Max.   :2019-07-16   Max.

# Output

## Export Wrangled DataFrame to CSV 
Export the full data set so that we can push it to the BQ database




In [ ]:
filename_final = "yvp_vegetation_cover_FINAL.csv"

if (filename_final %in% list.files(getwd())) {
  cat("file already exists in working directory:", filename_final, "\n", "working directory:", getwd(), "\n")
} else {
  write.csv(yvp_vegetation_cover_FINAL, row.names = FALSE, filename_final)
  cat(filename_final, "written to working directory \n", "working directory:", getwd(), "\n")
}

file already exists in working directory: yvp_vegetation_cover_FINAL.csv 
 working directory: /content 


## Push to BigQuery

"yvp_vegetation_cover_FINAL.csv" uploaded manually to BigQuery

## Export field datasheet version
Field datasheets need to have a complete, cumulative species list for each plot recorded in a table, with the cover_pct column set to 0. This allows field techs to change the 0 to some number if the species is found. The date column is blank so that field techs can fill in the appropriate date. Do not include columns that are needed for data analysis, like plot_loc, plot_rep, plot_num, and species_key. 

**Schema for field data sheet**

* plot_num (helps for sorting and finding plots)
* plot_code
* date
* species_code
* cover_pct

## Field data sheet production log
* Produced for 2020 field season

In [ ]:
field_datasheet = 
yvp_vegetation_cover_FINAL %>% 
select(plot_num, plot_code, subplot, species_code) %>% 
group_by(plot_num, plot_code, subplot) %>% 
distinct(species_code) %>% 
select(-species_code, species_code) %>% 
add_column(date = NA, .after = "plot_code") %>% 
add_column(cover_pct = 0) %>% 
arrange(plot_num, plot_code, subplot, species_code) %>% 
glimpse()

Rows: 8,861
Columns: 6
Groups: plot_num, plot_code, subplot [580]
$ plot_num     <int> 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,…
$ plot_code    <chr> "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP N7", "YVP N…
$ date         <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ subplot      <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,…
$ species_code <chr> "ACHMIL", "ALYALY", "BROTEC", "CAMMIC", "CARE_SP", "COLL…
$ cover_pct    <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…


In [ ]:
filename_field_datasheet = "yvp_vegetation_cover_field_datasheet_FINAL.csv"

if (filename_field_datasheet %in% list.files(getwd())) {
  cat("file already exists in working directory:", filename_final, "\n", "working directory:", getwd(), "\n")
} else {
  write.csv(field_datasheet, filename_field_datasheet)
  cat(filename_final, "written to working directory \n", "working directory:", getwd(), "\n")
}

ERROR: ignored